In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.preprocessing import LabelEncoder

# --- 1. 内存优化函数 (老朋友了) ---
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                else:
                    df[col] = df[col].astype(np.float32)
    return df

# --- 2. 读取数据 ---
print("正在读取数据...")
train_trans = pd.read_csv('../data/train_transaction.csv')
train_id = pd.read_csv('../data/train_identity.csv')
train_trans = reduce_mem_usage(train_trans)
train_id = reduce_mem_usage(train_id)
train = train_trans.merge(train_id, on='TransactionID', how='left')
del train_trans, train_id
gc.collect()

# --- 3. 特征工程 (刚才做的) ---
print("正在构造特征...")
# 3.1 时间特征
train['hour'] = (train['TransactionDT'] // 3600) % 24
# 3.2 聚合特征 (金额偏离度)
train['card1_amt_mean'] = train.groupby('card1')['TransactionAmt'].transform('mean')
train['amt_minus_mean'] = train['TransactionAmt'] - train['card1_amt_mean']
# 3.3 频次特征
train['card1_count'] = train.groupby('card1')['TransactionID'].transform('count')

print(f"✅ 数据准备完毕！形状: {train.shape}")

In [ ]:
# 找出所有是字符串 (object) 的列
cat_cols = list(train.select_dtypes(include=['object']).columns)

print(f"正在将 {len(cat_cols)} 个文本特征转化为数字...")

for col in cat_cols:
    # 填充空值，防止报错
    train[col] = train[col].fillna('unseen_before')
    
    # 转化
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    
    # 转完后变成数字了，顺便把类型转回 int，省内存
    train[col] = train[col].astype('int32')

print("✅ 所有特征已数字化，可以喂给模型了！")

In [ ]:
import lightgbm as lgb

# 1. 准备输入(X)和输出(y)
# 去掉 'isFraud' (答案), 'TransactionID' (无意义编号), 'TransactionDT' (时间戳本身)
X = train.drop(['isFraud', 'TransactionID', 'TransactionDT'], axis=1)
y = train['isFraud']

# 2. 按时间切分 (前80% vs 后20%)
split_idx = int(len(X) * 0.8)

X_train = X.iloc[:split_idx]
y_train = y.iloc[:split_idx]

X_val = X.iloc[split_idx:]
y_val = y.iloc[split_idx:]

print(f"训练集数量: {len(X_train)} (过去的数据)")
print(f"验证集数量: {len(X_val)} (未来的数据 - 用来考试)")

# 3. 构建 LightGBM 数据集对象
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

In [ ]:
# 设置参数 (工业界标准起手式)
params = {
    'objective': 'binary',      # 二分类任务
    'metric': 'auc',            # 评估指标：AUC
    'is_unbalance': True,       # 关键！告诉模型坏人很少，要重点关注
    'boosting_type': 'gbdt',    # 梯度提升树
    'num_leaves': 31,           # 树的复杂度
    'learning_rate': 0.05,      # 学习率
    'feature_fraction': 0.9,    # 每次只用90%的特征，防止过拟合
    'bagging_fraction': 0.8,    # 每次只用80%的数据
    'bagging_freq': 5,
    'verbose': -1
}

print("🚀 开始训练模型...")

clf = lgb.train(
    params,
    train_data,
    num_boost_round=1000,       # 最多跑1000轮
    valid_sets=[train_data, val_data], 
    callbacks=[
        lgb.early_stopping(stopping_rounds=50), # 如果50轮没进步就提前停止
        lgb.log_evaluation(period=50)           # 每50轮打印一次分数
    ]
)

print(f"🏆 最佳 AUC 分数: {clf.best_score['valid_1']['auc']:.4f}")

In [ ]:
# 1. 画出特征重要性排序 (Feature Importance)
# max_num_features=20: 只看最重要的前 20 个
lgb.plot_importance(clf, max_num_features=20, figsize=(10, 8), importance_type='split')
plt.title('Top 20 Features (Split Importance)')
plt.show()

# 2. (可选) 如果你想看具体的数值，可以打印出来
importance = pd.DataFrame({
    'feature': clf.feature_name(),
    'gain': clf.feature_importance(importance_type='gain')
}).sort_values(by='gain', ascending=False)

print(importance.head(10))